In [1]:
import numpy as np
import qutip as qt
from matplotlib import pyplot as plt
from QDSim import Simulation
from QDSim.noise import thermal_state, cavity_loss, thermal_in, relaxation, decoherence
from QDSim.physics import root_iSWAP
from QDSim.solvers import do_qt_mesolve

In [2]:
def get_lind_list(w_C, Qfactor, T, gamma_1, gamma_2, dims):
    cav_loss = cavity_loss(Qfactor, w_C, T, dims)
    therm_in = thermal_in(Qfactor, w_C, T, dims)
    d1 = decoherence(gamma_2, dims, 1)
    d2 = decoherence(gamma_2, dims, 2)
    r1 = decoherence(gamma_1, dims, 1)
    r2 = decoherence(gamma_1, dims, 2)
    return [cav_loss, therm_in, d1, d2, r1, r2]

In [3]:
sim = Simulation(do_qt_mesolve, fname='time_dependent.yaml')
state = qt.tensor(thermal_state(sim.w_c, 20e-3, sim.cav_dim),
                 qt.basis(2, 0), qt.basis(2, 1))
sim.set_state(state)

In [ ]:
q_factors = [1e5, 1e4, 1e3, 1e2, 10]
fids = []
target = root_iSWAP * qt.tensor(qt.basis(2, 0), qt.basis(2, 1))
for q in q_factors:
    ls = get_lind_list(sim.w_c, q, 100e-3, 1, 1e2, sim.dims)
    sim.lindblads = ls
    states = sim.run_solver(nsteps=1000, steps=5000, tau=1e-9)
    fids.append([qt.fidelity(target, s.ptrace([1,2])) for s in states])

In [ ]:
points = ['r-','g-','b-', 'y-', 'k-']
labels = ['100000','10000','1000','100','10']
times = 1e-9 * np.arange(5000)
fig = plt.figure()
ax = fig.add_subplot(111, title='Dispersive iSWAP Fidelity with changing Q Factor')
for i, fid in enumerate(fids):
    ax.plot(times, fid, points[i], label=labels[i])
ax.legend(loc='upper right')
ax.set_xlabel('Gate Time /s')
ax.set_ylabel('Fidelity')
plt.show()

In [5]:
points = ['r-','g-','b-', 'y-', 'k-']
times = 1e-9 * np.arange(5000)
labels = ['20mK', '50mK', '100mK', '250mK']
Ts = [20e-3, 50e-3, 100e-3, 250e-3]
q1 = qt.basis(2, 0)
q2 = qt.basis(2, 1)
fids = []
target = root_iSWAP * qt.tensor(qt.basis(2, 0), qt.basis(2, 1))
for T in Ts:
    cav = thermal_state(sim.w_c, T, sim.cav_dim)
    sim.set_state(qt.tensor(cav, q1, q2))
    ls = get_lind_list(sim.w_c, 1e4, T, 1, 1e2, sim.dims)
    sim.lindblads = ls
    states = sim.run_solver(nsteps=1000, steps=5000, tau=1e-9, progress_bar=True)
    fids.append([qt.fidelity(target, s.ptrace([1,2])) for s in states])
fig = plt.figure()
ax = fig.add_subplot(111, title='Dispersive iSWAP Fidelity with increasing T')
for i, fid in enumerate(fids):
    ax.plot(times, fid, points[i], label=labels[i])
ax.legend(loc='upper right')
ax.set_xlabel('Gate Time /s')
ax.set_ylabel('Fidelity')
plt.savefig('TSweep.pdf')

10.0%. Run time:   0.22s. Est. time left: 00:00:00:01
20.0%. Run time:   0.53s. Est. time left: 00:00:00:02
30.0%. Run time:   0.87s. Est. time left: 00:00:00:02
40.0%. Run time:   1.11s. Est. time left: 00:00:00:01
50.0%. Run time:   1.36s. Est. time left: 00:00:00:01
60.0%. Run time:   1.62s. Est. time left: 00:00:00:01
70.0%. Run time:   1.93s. Est. time left: 00:00:00:00
80.0%. Run time:   2.37s. Est. time left: 00:00:00:00
90.0%. Run time:   2.78s. Est. time left: 00:00:00:00
Total run time:   3.08s
10.0%. Run time:   0.27s. Est. time left: 00:00:00:02
20.0%. Run time:   0.58s. Est. time left: 00:00:00:02
30.0%. Run time:   0.87s. Est. time left: 00:00:00:02
40.0%. Run time:   1.18s. Est. time left: 00:00:00:01
50.0%. Run time:   1.48s. Est. time left: 00:00:00:01
60.0%. Run time:   1.81s. Est. time left: 00:00:00:01
70.0%. Run time:   2.30s. Est. time left: 00:00:00:00
80.0%. Run time:   2.73s. Est. time left: 00:00:00:00
90.0%. Run time:   3.04s. Est. time left: 00:00:00:00
Tota